In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, os, sys, shutil
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('./icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('./icr-identify-age-related-conditions/test.csv')
meta = pd.read_csv('./icr-identify-age-related-conditions/greeks.csv')

In [3]:
train_init = train.copy()
train['Alpha'] = meta['Alpha']
train['Beta'] = meta['Beta']
train['Gamma'] = meta['Gamma']
train['Delta'] = meta['Delta']

In [4]:
# 处理greeks
# Alpha为A的时候为0，其余为1
train['Alpha'] = train['Alpha'].apply(lambda x: 0 if x == 'A' else 1)

In [5]:
# 将数据集中唯一的离散特征转为01，这可能表明患者的性别
train['EJ'] = train['EJ'].replace({'A': 0, 'B': 1}).astype(float)
test['EJ']  = test['EJ'].replace({'A': 0, 'B': 1}).astype(float)
train_init['EJ'] = train_init['EJ'].replace({'A': 0, 'B': 1}).astype(float)
# 缺失值处理，用中位数填充
train['BQ'].fillna(0, inplace=True)
train.fillna(train.median(), inplace=True)
test.fillna(test.median(), inplace=True)
train_init.fillna(train_init.median(), inplace=True)
# 移除ID列
train_id = train['Id'].copy()
test_id  =  test['Id'].copy()
train = train.drop(['Id'], axis=1)
test  =  test.drop(['Id'], axis=1)

In [6]:
# 将gamma中的M和N都转为0，G，H，E，F，A，B的转为1
train['Gamma'] = train['Gamma'].replace({'M': 0, 'N': 0, 'G': 1, 'H': 1, 'E': 1, 'F': 1, 'A': 1, 'B': 1})
# 将Beta中的C0,B1,A2
train['Beta'] = train['Beta'].replace({'C': 0, 'B': 1, 'A': 2})
# 将Delta中的B0,A1,C1,D2
train['Delta'] = train['Delta'].replace({'B': 0, 'A': 1, 'C': 1, 'D': 2})

train.head()


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,72.611063,2003.810319,22.136229,69.834944,0.120343,1,1,0,1,2
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,72.611063,27981.562750,29.135430,32.131996,21.978000,0,0,0,0,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,0,0,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,0,0,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,146.109943,8524.370502,45.381316,36.262628,0.096614,1,1,1,1,0


In [7]:
# 连续变量归一化处理
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_columns = [_ for _ in train.columns if _ not in ['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Class']]
train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])
train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GE,GF,GH,GI,GL,Class,Alpha,Beta,Gamma,Delta
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,-0.410260,-0.655511,-0.948991,0.531241,-0.814049,1,1,0,1,2
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.410260,0.687893,-0.238862,-0.509218,1.304860,0,0,0,0,0
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,-0.299210,-0.051850,-0.351743,-0.424754,-0.806623,0,0,0,0,0
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,-0.342195,-0.650833,0.858232,1.101332,-0.810609,0,0,0,0,0
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.099920,-0.318309,1.409422,-0.395228,-0.816349,1,1,1,1,0


In [8]:
# 抛弃高度相关的特征
drop_cols = ['BZ','CL','EH','GL']  # 高度相关
drop_cols2 = ['DY','CB','GB','CH','DL','CU','FS','AZ','GE','EG','EP']  # 无用特征，同一点取得极值
drop_cols3 = ['BZ', 'DV', 'EH', 'FD ']
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [9]:
# ytrain为离散
ytrain = train[['Class', 'Alpha', 'Beta', 'Gamma', 'Delta']]
train.drop(['Class', 'Alpha', 'Beta', 'Gamma', 'Delta'], axis=1, inplace=True)

In [10]:
train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,-0.112922,-2.226608,0.162821,-0.035806,-0.250437,-0.940094,-0.410260,-0.655511,-0.948991,0.531241
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.311056,0.084542,-0.457925,-0.060566,0.113640,-1.145070,-0.410260,0.687893,-0.238862,-0.509218
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,-0.173811,0.517060,0.198663,-0.051023,0.597343,1.637944,-0.299210,-0.051850,-0.351743,-0.424754
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,0.058201,1.617160,0.060364,-0.060566,-0.104806,-0.219883,-0.342195,-0.650833,0.858232,1.101332
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.520331,1.212850,0.237302,0.896815,-0.229632,-0.432313,0.099920,-0.318309,1.409422,-0.395228


# 模型搭建
先直接对Class预测

In [11]:
# 数据集分割
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
x_train, x_test, y_train, y_test = train_test_split(train, ytrain, test_size=0.2, random_state=42)
scale_pos_weight = 4.712962962962963
print(x_train.shape)
print(y_train.shape)

(493, 52)
(493, 5)


In [12]:
def ScoreMetric(ytrue, ypred):
    nc = np.bincount(ytrue);
    return log_loss(ytrue, ypred, sample_weight = 1 / nc[ytrue], eps=1e-15);

In [13]:
from typing import Tuple
import xgboost as xgb
def balancedlogloss(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    ''' balanced log loss metric.'''
    y = dtrain.get_label()
    target_mean = y.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y]
    loss = log_loss(y, predt, eps = 1e-15, sample_weight=sample_weight)
    
    return 'balancedlogloss', loss

In [14]:
from lightgbm import LGBMClassifier

lgbmc = LGBMClassifier(learning_rate=0.005, num_iterations=775, force_col_wise=True)
lgbmc.fit(x_train, y_train['Class'])
lgbmc_pred = lgbmc.predict(x_test)
y_pred = lgbmc.predict_proba(x_test)
lgbmc_accuracy = accuracy_score(lgbmc_pred, y_test['Class'])

lgbmc_accuracy

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 85, number of negative: 408
[LightGBM] [Info] Total Bins 6769
[LightGBM] [Info] Number of data points in the train set: 493, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172414 -> initscore=-1.568616
[LightGBM] [Info] Start training from score -1.568616
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

0.9354838709677419

In [15]:
p0 = y_pred[:,1]
ScoreMetric(y_test['Class'], p0)
# p0

0.3576962361156256

# 正式的KFold交叉验证 + Optuna模型调参

In [16]:
import optuna, tune

from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, log_loss
from sklearn.metrics import make_scorer, accuracy_score, log_loss
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

# fold
from sklearn.model_selection import KFold, StratifiedKFold


In [19]:
def objective(trial):
    param = {
        "metric": "binary",
        "random_state": 42,
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 100, 1000),
        "n_estimators": trial.suggest_int("n_estimators", 700, 10000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 300, step=3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 0.7),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 0.7),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.8, 1, step=0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=5),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.9),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0)
    }

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    log_loss_scores = []
    # 划分验证集
    for train_index, val_index in skf.split(train, ytrain['Class']):
        x_train_, x_val = train.iloc[train_index], train.iloc[val_index]
        y_train_, y_val = ytrain['Class'].iloc[train_index], ytrain['Class'].iloc[val_index]

        classifier = LGBMClassifier(**param, scale_pos_weight=scale_pos_weight)
        classifier.fit(x_train_, y_train_, eval_set=[(x_val, y_val)])

        y_pred_proba = classifier.predict_proba(x_val)[:, 1]
        log_loss_scores.append(ScoreMetric(y_val, y_pred_proba))

    return np.mean(log_loss_scores)


if __name__ == "__main__":
    sampler = TPESampler(seed=42)
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
    study = optuna.create_study(pruner=pruner, direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=500)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-08-03 11:52:56,657] A new study created in memory with name: no-name-bb63a5f0-a048-4297-a8f5-10264d1eaf4f
[W 2023-08-03 11:52:56,668] Trial 0 failed with parameters: {'early_stopping_rounds': 437, 'n_estimators': 9542, 'learning_rate': 0.22227824312530747, 'num_leaves': 184, 'max_depth': 4, 'reg_alpha': 0.11763621903197981, 'reg_lambda': 0.050077692396057624, 'bagging_fraction': 0.9000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.9, 'min_child_samples': 75, 'colsample_bytree': 0.11646759543664197, 'lambda_l1': 5.360294728728285, 'lambda_l2': 0.31044435499483225, 'subsample': 0.29110519961044856} because of the following error: TypeError("fit() got an unexpected keyword argument 'verbose'").
Traceback (most recent call last):
  File "c:\Users\111\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\111\AppData\Local\Temp\ipykernel_14884\2076078773.py", line 30, in 

TypeError: fit() got an unexpected keyword argument 'verbose'

In [ ]:
def lgb_metric(y_true, y_pred):
    loss = ScoreMetric(y_true, y_pred)
    return 'lgb_metric', loss, False

In [ ]:
model = LGBMClassifier(**trial.params)
model.fit(x_train, y_train['Class'], eval_metric=lgb_metric)
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)[:, 1]
ScoreMetric(y_test['Class'], y_pred_proba)
accuracy_score(y_pred, y_test['Class'])

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_param_importances(study)

In [ ]:
trial.params

{'early_stopping_rounds': 116,
 'n_estimators': 8594,
 'learning_rate': 0.2292825799916429,
 'num_leaves': 190,
 'max_depth': 3,
 'reg_alpha': 0.3086813444028655,
 'reg_lambda': 0.08439961817618014,
 'bagging_fraction': 1.0,
 'bagging_freq': 1,
 'feature_fraction': 1.0,
 'min_child_samples': 75,
 'colsample_bytree': 0.2572293361418775,
 'lambda_l1': 1.246275770846192e-06,
 'lambda_l2': 0.011660417895786973,
 'subsample': 0.7301110313724658}

 {'n_estimators': 924,
 'learning_rate': 0.25697226732130196,
 'num_leaves': 85,
 'max_depth': 3,
 'reg_alpha': 0.31540771886764585,
 'reg_lambda': 0.2432782070795778,
 'bagging_fraction': 1.0,
 'bagging_freq': 1,
 'feature_fraction': 0.8,
 'min_child_samples': 75,
 'colsample_bytree': 0.2801457679018723,
 'lambda_l1': 0.0033213165525154092,
 'lambda_l2': 5.2592633826837275e-05,
 'subsample': 0.86834033201949}

 